Import dependencies

In [3]:
import landmark_detector as ld
import os
import numpy as np
import keras
from keras import layers

Set variables

In [4]:
words = ['deaf', 'eat', 'fish', 'friend', 'like', 'milk', 'nice', 'no', 'orange', 'teacher', 'want', 'what', 'where', 'yes']
select_words = ['no', 'eat', 'teacher']
modes = ['train', 'val', 'test']
path = '../preprocessing/dataset/'
num_features = 126
model_name = 'draft_model'

Get data from mediapipe


In [5]:
def get_data(mode, words, path, detector_path):
    detector = ld.get_detector(detector_path)

    training_X = []
    training_y = []

    num_videos = 0
    highest_frame = 0

    for word in words:
        i = 1
        video_path = path + mode + '/' + word + '/0001.mp4'
        while os.path.exists(video_path):
            try:
                video_X = []
                landmarks, current_frames = ld.get_landmarks(video_path, detector)
                if len(landmarks) == 0:
                    i += 1
                    video_path = path + mode + '/' + word + '/' + str(i).zfill(4) + '.mp4'
                    continue
                if current_frames > highest_frame:
                    highest_frame = current_frames
                for frame in range(len(landmarks)):
                    features = np.array(landmarks[frame]).flatten()
                    video_X.append(features)
                training_X.append(video_X)
                training_y.append(words.index(word))
                i += 1
                num_videos += 1
                video_path = path + mode + '/' + word + '/' + str(i).zfill(4) + '.mp4'
            except Exception as e:
                print(e)
                break

    return training_X, training_y, num_videos, highest_frame

training_X, training_y, num_videos, highest_frame = get_data('train', select_words, path, '../models/hand_landmarker.task')

print('Number of videos:', num_videos)
print('Highest frame:', highest_frame)

Number of videos: 91
Highest frame: 47


Padding and Masking X

In [6]:
def padX(X, num_videos, highest_frame, num_features):
    padded_X = np.zeros((num_videos, highest_frame, num_features))
    mask = np.ones((num_videos, highest_frame, num_features)) 
    for i in range(num_videos):
        video = X[i]
        for j in range(len(video)):
            frame = video[j]
            if len(frame) < num_features:
                padded_X[i, j, :] = np.pad(frame, (0, num_features - len(frame)), 'constant')
                mask[i, j, len(frame):] = 0
            else:
                padded_X[i, j, :] = frame
        if len(video) < highest_frame:
            mask[i, len(video):, :] = 0

    return padded_X, mask

padded_X, mask = padX(training_X, num_videos, highest_frame, num_features)
print(padded_X.shape)

(91, 47, 126)


Create model

In [7]:
model = keras.Sequential()

model.add(keras.Input(shape=(highest_frame, num_features)))
#model.add(keras.layers.SimpleRNN(len(select_words), activation='relu'))
model.add(layers.Masking(mask_value=0.0))
model.add(layers.LSTM(64))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(len(select_words), activation='softmax'))


model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 47, 126)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,075 (199.51 KB)

 Trainable params: 51,075 (199.51 KB)

 Non-trainable params: 0 (0.00 B)

Train model

In [8]:
val_X, val_y, num_val_videos, highest_frame_val = get_data('val', select_words, path, '../models/hand_landmarker.task')
padded_val_X, val_mask = padX(val_X, num_val_videos, highest_frame, num_features)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(padded_X, np.array(training_y), epochs=10, validation_data=(padded_val_X, np.array(val_y)))

model.save(f'../models/{model_name}.keras')

with open(f"../models/{model_name}.env", "w") as file:
    file.write(f"MAX_FRAMES={highest_frame}\n")
    file.write(f"NUM_FEATURES={num_features}\n")
    file.write(f"WORDS={",".join(select_words)}\n")

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 188ms/step - accuracy: 0.3360 - loss: 1.1232 - val_accuracy: 0.4167 - val_loss: 1.0779
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.5082 - loss: 1.0473 - val_accuracy: 0.5833 - val_loss: 1.0549
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.5428 - loss: 1.0032 - val_accuracy: 0.5000 - val_loss: 1.0279
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.5576 - loss: 0.9723 - val_accuracy: 0.5000 - val_loss: 1.0018
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6101 - loss: 0.9365 - val_accuracy: 0.5417 - val_loss: 0.9743
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5483 - loss: 0.9182 - val_accuracy: 0.5833 - val_loss: 0.9476
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.6312 - loss: 0.8389 - val_accuracy: 0.6250 - val_loss: 0.9258
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.6679 - loss: 0.7824 - val_accuracy: 0.6250 - val_loss: 0.9039

Evaluate model

In [9]:
test_X, test_y, num_test_videos, highest_frame_test = get_data('test', select_words, path, '../models/hand_landmarker.task')
padded_test_X, test_mask = padX(test_X, num_test_videos, highest_frame, num_features)
results = model.evaluate(padded_test_X, np.array(test_y))

print('Test loss:', results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7333 - loss: 0.7408
Test loss: [0.7407840490341187, 0.7333333492279053]


Use model

In [10]:
detector = ld.get_detector('../models/hand_landmarker.task')

prediction_X = []
prediction_y = []

video_path = path + 'test/teacher/0004.mp4'

video_X = []
landmarks, frame_count = ld.get_landmarks(video_path, detector)
if len(landmarks) == 0:
    print('No landmarks detected')
else:
    for frame in range(len(landmarks)):
        features = np.array(landmarks[frame]).flatten()
        features = np.pad(features, (0, num_features - len(features)), 'constant')
        video_X.append(features)
    for i in range(highest_frame-len(video_X)):
        temp = np.zeros((num_features))
        video_X.append(temp)

    prediction_X.append(video_X)
    prediction_y.append(select_words.index('teacher'))


    print(np.shape(prediction_X))
    print(model.predict(np.array(prediction_X)))

(1, 47, 126)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
[[0.00329665 0.01375335 0.98295   ]]
